<h1 align="center">fastText for text classification</h1>

***

In this notebook, we will train a fastText model for criteria sentence classification, and evalute the performance in test data.

**What is fastText?**
fastText is a library for efficient learning of word representations and sentence classification.

* training data (22962 sentences), validation data (7682 sentences) test data (7697 sentences)
* 44 semantic categories

|#|group topics|semantic categories|
|---|---|----
|1|`Health Status`|`Disease` `Symptom` `Sign` `Pregnancy-related Activity` `Neoplasm Status` `Non-Neoplasm Disease Stage` `Allergy Intolerance` `Organ or Tissue Status` `Life Expectancy` `Oral related`
|2|`Treatment or Health Care`|`Pharmaceutical Substance or Drug` `Therapy or Surgery` `Device` `Nursing`
|3|`Diagnostic or Lab Test`|`Diagnostic` `Laboratory Examinations` `Risk Assessment` `Receptor Status`
|4|`Demographic Characteristics`|`Age` `Special Patient Characteristic` `Literacy` `Gender` `Education` `Address` `Ethnicity`
|5|`Ethical Consideration`|`Consent` `Enrollment in other studies` `Researcher Decision` `Capacity` `Ethical Audit` `Compliance with Protocol`
|6|`Lifestyle Choice`|`Addictive Behavior` `Bedtime` `Exercise` `Diet` `Alcohol Consumer` `Sexual related` `Smoking Status` `Blood Donation`
|7|`Data or Patient Source`|`Encounter` `Disabilities` `Healthy` `Data Accessible`
|8|`Other`|`Multiple`

In [1]:
import os
import sys
import fasttext
import codecs
import jieba

<h2>Getting and preparing the data</h2>

***

Before training our first classifier, we need to prepare the train data and test data. We will use the test data to evaluate how good the learned classifier is.

Each line of the text file contains a list of labels, followed by the corresponding sentence. All the labels start by the _ _label_ _ prefix, which is how fastText recognize what is a label or what is a word. The model is then trained to predict the labels given the word in the document.

train data:

In [2]:
with open("criteria.train", "w", encoding="utf-8") as outf:
    with open("../data/train.txt", "r", encoding="utf-8") as inf:
        for line in inf:
            l = line.strip().split("\t")
#             sentence = jieba.cut(l[2].strip().replace("\t", " ").replace("\n", " "))
            sentence = [w for w in l[2].strip().replace("\t", " ").replace("\n", " ")]
            outf.write("__label__{} {}\n".format(l[1].replace(" ", "_"), " ".join(list(sentence))))

test data:

In [3]:
with open("criteria.test", "w", encoding="utf-8") as outf:
    with open("../data/test.txt", "r", encoding="utf-8") as inf:
        for line in inf:
            l = line.strip().split("\t")
#             sentence = jieba.cut(l[2].strip().replace("\t", " ").replace("\n", " "))
            sentence = [w for w in l[2].strip().replace("\t", " ").replace("\n", " ")]
            outf.write("__label__{} {}\n".format(l[1].replace(" ", "_"), " ".join(list(sentence))))

<h2>fastText classifier</h2>

***

Automatic hyperparameter optimization

In [4]:
model = fasttext.train_supervised(input="criteria.train",autotuneValidationFile='criteria.train')

In [5]:
model.test("criteria.test")

(7697, 0.8182408730674289, 0.8182408730674289)

<h2>Save model and test data results</h2>

***

call save_model to save it as a file.

In [6]:
model.save_model("fastText_criteria.bin")

load model with load_model function, and evaluate on test data.

In [7]:
test_data_file = "criteria.test"
test_results_save_file = "fasttext_test_pred.txt"

criteria_ids, criteria_sentences = [], []
with open(test_data_file, "r", encoding="utf-8") as inf:
    c = 0
    for line in inf:
        c += 1
        l = line.strip().split(" ")
        criteria_ids.append("s{}".format(c))
        criteria_sentences.append(" ".join(l[1:]))
        
model = fasttext.load_model("fastText_criteria.bin")        
predicted = model.predict(criteria_sentences, k=1)

with codecs.open(test_results_save_file, "w", encoding="utf-8") as outf:
    for i in range(len(criteria_ids)):
        outf.write("{}\t{}\t{}\n".format(criteria_ids[i], predicted[0][i][0].replace("__label__", "").replace("_", " "), "".join(criteria_sentences[i].split())))

<h2>Evaluation</h2>

***

In [8]:
import evaluation # our defined evaluation metrics.
results = evaluation.Record_results('../data/test.txt', 'fasttext_test_pred.txt')
evaluation = evaluation.Evaluation(results.records)

**************************************** Evaluation results*****************************************
                                       Precision.       Recall.          f1.            
                 Addictive Behavior    0.8987           0.8068           0.8503         
                            Address    0.6154           0.6667           0.6400         
                                Age    0.9803           0.9770           0.9787         
                   Alcohol Consumer    0.6250           0.8333           0.7143         
                Allergy Intolerance    0.9318           0.9193           0.9255         
                            Bedtime    0.7778           0.5833           0.6667         
                     Blood Donation    0.7692           0.9091           0.8333         
                           Capacity    0.5926           0.5714           0.5818         
           Compliance with Protocol    0.7869           0.8000           0.7934         
         

<h2>Predict a new input criteria sentence with saved model</h2>

***

In [11]:
examples = ["性别不限", "年龄大18岁，", "病人对研究药物过敏。"]
print(examples)
sentences = [" ".join([w for w in s.strip().replace("\t", " ").replace("\n", " ")]) for s in examples]
# [" ".join([s.strip().replace("\t", " ").replace("\n", " "))) for s in examples))]
print(sentences)

model = fasttext.load_model("fastText_criteria.bin") 
results = model.predict(sentences, k=1)
print(results)

['性别不限', '年龄大18岁，', '病人对研究药物过敏。']
['性 别 不 限', '年 龄 大 1 8 岁 ，', '病 人 对 研 究 药 物 过 敏 。']
([['__label__Gender'], ['__label__Age'], ['__label__Allergy_Intolerance']], [array([0.9999449], dtype=float32), array([1.00001], dtype=float32), array([1.0000079], dtype=float32)])
